In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy.linalg as la

## Funciones

Auxiliares

In [54]:
def hess(f, x):
    '''
        Input:
            f: lambda function
            x: function args
        Output:
            hess_f: hessian of f at x
    '''
    n = len(x)
    hess_f = np.zeros([n, n])
    
    E = np.diag([pow(np.finfo(float).eps, 1/4) * (abs(a) + 1) for a in x])
    
    for i in range(n):
        for j in range(n):
            hess_f[i, j] = (  f(x + E[:, i] + E[:, j]) 
                            - f(x - E[:, i] + E[:, j]) 
                            - f(x + E[:, i] - E[:, j]) 
                            + f(x - E[:, i] - E[:, j]) ) * (0.25 / (E[i, i] * E[j, j]))
    
    return hess_f
    

In [40]:
def grad(f, x):
    '''
        Input:
            f: lambda function
            x: function args
        Output:
            grad_f: function gradient at x
    '''
    n = len(x)
    grad_f = np.zeros(n)
    
    E = np.diag([pow(np.finfo(float).eps, 1/3) * (abs(a) + 1) for a in x])
    
    for i in range(n):
        grad_f[i] = (f(x + E[:, i]) - f(x - E[:, i])) * (0.5 / E[i, i])
    
    return grad_f

In [46]:
def getCoorDir(grad_k):
    pos = abs(grad_k).argmax()
    
    return np.array([-1 if i == pos else 0 for i in range(len(grad_k))])

In [42]:
def bisecc(f, x_k, d_k, slope):
    alpha_k = 1
    f_k = f(x_k)
    
    while f(x_k + alpha_k*d_k) > f_k + alpha_k*slope:
        alpha_k = alpha_k/2
        
    return alpha_k

Descenso por coordenada

In [43]:
def desCoor(f, x0, tol, maxiter):
    """
    Descends in one variable per iteration
    
    params:
        f       - function to minimize
        x0      - initial point
        tol     - tolerance
        maxiter - upper bound for iterations
        
    returns:
        xf   - final approximation of x*
        iter - # of iterations used
    """
    
    i = 0
    
    c1 = 0.1
    x_k = x0
    
    
    while i < maxiter:
        grad_k = grad(f, x_k)
        
        if la.norm(grad_k, ord = np.inf) < tol:
            break
        
        d_k = getCoorDir(grad_k)
        
        slope = c1 * np.dot(grad_k, d_k)
        alpha_k = bisecc(f, x_k, d_k, slope)
        
        x_k = x_k + alpha_k * d_k
        
        i += 1
        
    return x_k, i

Descenso por Newton

In [57]:
# 15 iteraciones
# x0 = [2,3]

def desNewton(f, x0, tol, maxiter):
    
    i = 0
    c1 = 0.1
    x_k = x0
    
    while i < maxiter:
        grad_k = grad(f, x_k)

        if la.norm(grad_k, ord = np.inf) < tol:
            break

        hess_k = hess(f, x_k)
        
        d_k = la.lstsq(-grad_k, hess_k)[0]
        
        slope = c1 * np.dot(grad_k, d_k)
        alpha_k = bisecc(f, x_k, d_k, slope)
        
        x_k = x_k + alpha_k * d_k
        
        i += 1
        
    return x_k, i

## Experimentos

#### Ejercicio 1

In [44]:
A = la.pascal(4)
b = np.ones(4)
f = lambda x : 1/2 * np.dot(x, np.dot(A, x)) + np.dot(b, x) + 1
#grad_f = lambda x : np.dot(A, x) + b
#hess_f = lambda x : A

x0 = np.array([1, 2, 3, 4])
tol = 1e-5
maxiter = 2000

In [47]:
desCoor(f, x0, tol, maxiter)

(array([ 1.,  0.,  3., -2.]), 2000)